# Домашнее задание

1) Попытаться ответить на вопросы/выдвинуть гипотезы

2) Доделать прошлые домашния задания

3) Прочитать статьи BM25/MatrixFactorization

4) Поэкспериментировать с ALS (grid-search)

---

In [1]:
# Загрузка необходимых библиотек

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.sparse import csr_matrix

from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

import os, sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from metrics import precision_at_k, recall_at_k

In [2]:
# Загрузка данных retail

data = pd.read_csv('retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns = {'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace = True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

# Проверка

data_train.head(5)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [3]:
# Загрузка данных product

item_features = pd.read_csv('product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns = {'product_id': 'item_id'}, inplace = True)

# Проверка

item_features.head(5)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ


In [4]:
item_features.department.unique()

array(['GROCERY', 'MISC. TRANS.', 'PASTRY', 'DRUG GM', 'MEAT-PCKGD',
       'SEAFOOD-PCKGD', 'PRODUCE', 'NUTRITION', 'DELI', 'COSMETICS',
       'MEAT', 'FLORAL', 'TRAVEL & LEISUR', 'SEAFOOD', 'MISC SALES TRAN',
       'SALAD BAR', 'KIOSK-GAS', 'ELECT &PLUMBING', 'GRO BAKERY',
       'GM MERCH EXP', 'FROZEN GROCERY', 'COUP/STR & MFG', 'SPIRITS',
       'GARDEN CENTER', 'TOYS', 'CHARITABLE CONT', 'RESTAURANT', 'RX',
       'PROD-WHS SALES', 'MEAT-WHSE', 'DAIRY DELI', 'CHEF SHOPPE', 'HBC',
       'DELI/SNACK BAR', 'PORK', 'AUTOMOTIVE', 'VIDEO RENTAL', ' ',
       'CNTRL/STORE SUP', 'HOUSEWARES', 'POSTAL CENTER', 'PHOTO', 'VIDEO',
       'PHARMACY SUPPLY'], dtype=object)

In [5]:
# Группирование для result

result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns = ['user_id', 'actual']

# Проверка

result.head(5)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."


In [6]:
# Группирование для popularity

popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns = {'quantity': 'n_sold'}, inplace = True)

top_5000 = popularity.sort_values('n_sold', ascending = False).head(5000).item_id.tolist()

In [7]:
# Ввод feature item_id

data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999_999

user_item_matrix = pd.pivot_table(data_train, 
                                  index = 'user_id', columns = 'item_id', 
                                  values = 'quantity',
                                  aggfunc = 'count', 
                                  fill_value = 0
                                 )
# Подготовка для implicit

user_item_matrix = user_item_matrix.astype(float)

# Sparse matrix

sparse_user_item = csr_matrix(user_item_matrix)

user_item_matrix.head(5)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

---

### ALS

In [9]:
def fit_model(factors=64, regularization=0.05, u_i_matrix='original'):

    all_variants_matrix = {
        'original' : user_item_matrix,
        'tfidf': tfidf_weight(user_item_matrix.T).T,
        'bm25': bm25_weight(user_item_matrix.T).T
    }

    model = AlternatingLeastSquares(factors=factors,
                                    regularization=regularization,
                                    iterations=15,
                                    calculate_training_loss=True,
                                    num_threads=4,
                                    use_gpu=False)

    model.fit(csr_matrix(all_variants_matrix[u_i_matrix]).T.tocsr(),  # На вход item-user matrix
              show_progress=True)

    return model

In [10]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)]
    return res

In [11]:
def grid_search(factors = None, regularizations = None, user_item_matrixes = None):
    global result
    if factors is None:
        factors = [64]
    if regularizations is None:
        regularizations = [0.05]
    if user_item_matrixes is None:
        user_item_matrixes = ['original']
    scores = pd.DataFrame()
    for factor in factors:
        for regularization in regularizations:
            for matrix in user_item_matrixes:

                name_test = f'matrix: {matrix}, factor: {factor}, regularization: {regularization}'

                model = fit_model(factors = factor, regularization = regularization, u_i_matrix = matrix)

                result[name_test] = result['user_id'].apply(lambda x: get_recommendations(x, model = model, N = 5))

                score = result.apply(lambda row: precision_at_k(row[name_test], row['actual']), axis = 1).mean()
                scores = scores.append(pd.DataFrame([score], columns = ['score'], index = [name_test]))
                print('score:', round(score,4), name_test)

    return result, scores.sort_values(by = 'score', ascending = False)

In [12]:
%%time

result, scores = grid_search(factors = [32, 64, 128],
                             regularizations = [0.01, 0.03, 0.05],
                             user_item_matrixes = ['original', 'tfidf', 'bm25'])

  0%|          | 0/15 [00:00<?, ?it/s]

ValueError: user_items must contain 1 row for every user in userids

**Результат:** 

- Следующим шагом был бы выбор лучшей метрики при определенных гиперпараметрах
- Для того, чтобы ответить на поставленные вопросы, необходимо разработать MVP. После обсуждения с менеджером и после первых итераций на пользователях производим продолжающиеся улучшения

---

# Links

BM25

https://en.wikipedia.org/wiki/Okapi_BM25#:~:text=BM25%20is%20a%20bag%2Dof,slightly%20different%20components%20and%20parameters.


Matrix factorization (ALS, SVD)
https://datasciencemadesimpler.wordpress.com/tag/alternating-least-squares/